In [ ]:
from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset_baby/'
sequence = '00'
image_instances_path = 'pipeline_baby/vfm-labels/sam/00/'
kitti = KittiDataset(dataset_path, sequence, image_instances_path)

In [ ]:
from services.preprocessing.common.config import ConfigDTO

config = ConfigDTO(**{
    'dataset': kitti,
    'start_index': 19,
    'end_index': 23,
    'start_image_index_offset': 3,
    'cam_name': 'cam2',
    'R': 12,
    'nb_neighbors': 30,
    'std_ratio': 5.0,
    'voxel_size': 0.25
})

In [ ]:
from services.preprocessing.init.map import InitMapProcessor
from services.preprocessing.init.instances_matrix import InitInstancesMatrixProcessor

init_pcd = InitMapProcessor().process(config)
points2instances = InitInstancesMatrixProcessor().process(config, init_pcd)

In [ ]:
from services.preprocessing.not_zero import SelectionNotZeroProcessor
from services.preprocessing.in_cube import SelectionInCubeProcessor
from services.preprocessing.statistical_outlier import StatisticalOutlierProcessor

processors = [
    SelectionNotZeroProcessor(),
    SelectionInCubeProcessor(),
    StatisticalOutlierProcessor()
]

In [ ]:
import copy

pcd = copy.deepcopy(init_pcd)
for processor in processors:
    pcd, points2instances = processor.process(config, pcd, points2instances)

pcd_for_clustering = copy.deepcopy(pcd)

In [ ]:
from services.preprocessing.voxel_down import VoxelDownProcessor

pcd, points2instances, voxel_results = VoxelDownProcessor().process(config, pcd, points2instances)
trace = voxel_results[2]

In [ ]:
import numpy as np

from scipy.spatial.distance import cdist
from utils.distances_utils import sam_label_distance

points = np.asarray(pcd.points)
spatial_distance = cdist(points, points)

dist, masks = sam_label_distance(points2instances, spatial_distance, 2, 10, 2)

In [ ]:
from utils.distances_utils import remove_isolated_points

dist, points, trace = remove_isolated_points(dist, points, trace)

In [ ]:
from services.normalized_cut_service import normalized_cut

T = 0.2
eigenval = 2
clusters = normalized_cut(dist, np.array([i for i in range(len(points))], dtype=int), T, eigenval)

In [ ]:
len(clusters)

In [ ]:
from utils.pcd_utils import color_pcd_by_clusters_and_voxels

pcd_clustered = color_pcd_by_clusters_and_voxels(pcd_for_clustering, trace, clusters)

In [ ]:
import open3d as o3d

o3d.visualization.draw_geometries([pcd_clustered])

In [ ]:
executed_id = 1
name = "labeled_cloud_modi_dist/clouds_{}_{}_R{}_voxel{}_T{}_eigenval{}_nb_neighbors{}_std_ratio{}_offset{}_executed_id{}".format(
    config.start_index, config.end_index, config.R, config.voxel_size, T, eigenval, 
    config.nb_neighbors, config.std_ratio, config.start_image_index_offset, executed_id
)
filename = name.replace(".", "l") + ".pcd"

o3d.io.write_point_cloud(filename, pcd_clustered)